In [3]:
import pandas as pd
import numpy as np
import dask.dataframe as dd
from surprise import Reader
from surprise import Dataset
from surprise import SVD
from surprise.model_selection import cross_validate
from surprise.model_selection import GridSearchCV

from collections import defaultdict

In [65]:
def clear_data(df:pd.DataFrame, min_num_purchase=10, store_position=50):
    print('--------------------------------------------------------------------------')
    print('Dataframe clean starts')
    print('--------------------------------------------------------------------------')

    # getting an id store placed on :store_position place in store list sorted by purchases
    # получение идентификатора магазина, размещенного на месте :store_position в списке магазинов, отсортированном по покупкам
    store = df['store_id'].value_counts().index[store_position]
        
    # getting purchasing data by selected store
    # получение данных о покупках по выбранному магазину
    one_store_data = df[df['store_id'] == store]
    
    print(f'Data collected for the store ranked {store_position} in terms of sales')
    print(f'Данные по магазину, занимающему {store_position}-е место по кол-ву продаж, собраны')
    print('--------------------------------------------------------------------------')
    
    users_by_purchases = one_store_data['user_id'].value_counts().reset_index()
    users_by_purchases = users_by_purchases.rename(columns={'index':'user_id', 'user_id':'purchases'})
    users_id_list = users_by_purchases[users_by_purchases['purchases'] > min_num_purchase]['user_id']
    users_filter = one_store_data['user_id'].isin(users_id_list)
    cleaned_data = one_store_data[users_filter]

    return cleaned_data

In [74]:
%%time
clear_data(raw_data)

--------------------------------------------------------------------------
Dataframe clean starts
--------------------------------------------------------------------------
Data collected for the store ranked 50 in terms of sales
Данные по магазину, занимающему 50-е место по кол-ву продаж, собраны
--------------------------------------------------------------------------
Wall time: 525 ms


,period,user_id,store_id,item_code,license,type_by_nomenclature,raiting
7615,2019-01-13 19:00:39,3000003653873,IZ-000067,00045063,True,C,4
24717,2019-05-02 19:06:19,3000000866184,IZ-000067,00153277,False,C,1
24718,2019-05-02 19:06:19,3000000866184,IZ-000067,00178667,False,B,2
24719,2019-05-02 19:06:19,3000000866184,IZ-000067,00022688,True,B,4
24720,2019-05-02 19:06:19,3000000866184,IZ-000067,00011101,True,A,4
...,...,...,...,...,...,...,...
2495579,2022-04-30 19:00:13,3000006376731,IZ-000067,00205347,False,B,3
2495580,2022-04-30 19:00:13,3000006376731,IZ-000067,00226159,False,B,4
2495581,2022-04-30 19:00:13,3000006376731,IZ-000067,00186392,False,A,4
2495582,2022-04-30 19:00:13,3000000838860,IZ-000067,00002376,False,C,4


In [81]:
def data_preparation(df: pd.DataFrame):
    print('Подготовка trainset')
    print('--------------------------------------------------------------------------')

    train_df = df[['user_id', 'item_id', 'rating']].reset_index(drop=True)

    # A reader is still needed but only the rating_scale param is requiered.
    reader = Reader(rating_scale=(1, 10))
    train_df = Dataset.load_from_df(train_df[['user_id', 'item_id', 'rating']], reader)
    return train_df

In [87]:
def clear_one_buy_client(df: pd.DataFrame, n_buy=10) -> pd.DataFrame:
    """ Функция очищает датафрейм от клиентов,
      которые сделали меньше n покупок

      parameters:
      df (pd.DataFrame): датафрейм, который нужно очистить
      n (int): Число покупок, по которому отсеиваются клиенты

      return:
      pd.DataFrame: очищенный датафрейм
    """
    a = df['user_id'].value_counts().reset_index()
    df_less_10 = a[a['user_id'] > n_buy]
    df_less_10 = df_less_10[['index']].rename(columns={'index': 'user_id'})
    return df_less_10.merge(df)

In [6]:
def lighter_data(df: pd.DataFrame, store_n=50, n_buy=10) -> pd.DataFrame:
    """
    :param df:
    :param store_n:
    :param n_buy:
    :return:
    """
    # выбор магазина для генерации рекомендаций
    rozn_rec_data_s1 = store_sort_df(df, store_n=store_n)
    rozn_rec_data_s1 = clear_one_buy_client(rozn_rec_data_s1, n_buy=n_buy)
    return rozn_rec_data_s1

In [38]:
def store_sort_df(df: pd.DataFrame, store_n=50) -> pd.DataFrame:
    """

    :param df:
    :param store_n:
    :return:
    """

    print('--------------------------------------------------------------------------')
    print('Подготовка датафрейма')
    print('--------------------------------------------------------------------------')

    # магазин с наибольшим числом покупок
    store = df['store_id'].value_counts().index[store_n]
    # Оставляем данные только по одному магазину
    rozn_rec_data_s1 = df[df['store_id'] == store]

    print(f'Данные по магазину, занимающему {store_n}-е место по кол-ву продаж, собраны')
    print('--------------------------------------------------------------------------')

    return rozn_rec_data_s1

In [4]:
def data_preparation(df: pd.DataFrame) -> pd.DataFrame:
    """

    :param df:
    :return:
    """
    print('Подготовка trainset')
    print('--------------------------------------------------------------------------')

    data_prep = df[['user_id', 'КодНоменклатуры', 'rating']].reset_index(drop=True)
    data_prep = data_prep.rename(columns={'user_id': 'userID', 'КодНоменклатуры': 'itemID'})

    # A reader is still needed but only the rating_scale param is requiered.
    reader = Reader(rating_scale=(1, 10))
    data_prep = Dataset.load_from_df(data_prep[['userID', 'itemID', 'rating']], reader)
    return data_prep

In [ ]:
def gssv(data_prep_df: pd.DataFrame, alg: type, param_grid: dict, measures, opt_by="rmse"):
    """

    :param data_prep_df:
    :param alg:
    :param param_grid:
    :param measures:
    :param opt_by:
    :return:
    """
    print('Проводится GridSearch')

    gs = GridSearchCV(alg, param_grid, measures=measures, cv=3)
    gs.fit(data_prep_df)

    print('--------------------------------------------------------------------------')
    # best RMSE score
    print(f'best_score {opt_by}: {gs.best_score[opt_by]}')
    # combination of parameters that gave the best RMSE score
    print(f'best_params {opt_by}: {gs.best_params[opt_by]}')

    print('--------------------------------------------------------------------------')

    return gs

In [12]:
def make_rec(data_prep_df: pd.DataFrame, n_pred=10,
             gridsearch=True, alg: type = SVD,
             opt_by='rmse'):
    """

    :param data_prep_df:
    :param n_pred:
    :param gridsearch:
    :param alg:
    :param opt_by:
    :return:
    """
    measures = ['RMSE', 'MAE']
    # тренировочные данные
    trainset = data_prep_df.build_full_trainset()

    # Опциональнй гридсёрч по заданным параметрам
    if gridsearch:
        # подбор оптимальных параметров модели by gridsearch
        param_grid = {'n_epochs': [10, 20, 30], 'lr_all': [0.002, 0.005, 0.01],
                      'reg_all': [0.02, 0.4, 0.6]}

        gs = gssv(alg=SVD, param_grid=param_grid,
                  measures=measures,
                  data_prep_df=data_prep_df, opt_by=opt_by)

        # We can now use the algorithm that yields the best rmse:
        algo = gs.best_estimator['rmse']
        algo.fit(trainset)

    else:
        # обучение алгоритма
        algo = alg()  # KNNBasic()
        algo.fit(trainset)

        print('Проводится Кроссвалидация')
        print('--------------------------------------------------------------------------')

        # кроссвалидация
        cross_validate(algo, data_prep_df, measures=measures, cv=4, verbose=True)

    # Подготовка тестового датасета
    testset = trainset.build_anti_testset()

    print('--------------------------------------------------------------------------')
    print('Генерация рекомендаций')
    print('--------------------------------------------------------------------------')

    # прогноз для тестового датасета
    # Than predict ratings for all pairs (u, i) that are NOT in the training set.
    predictions = algo.test(testset)

    # Сохраняем топ-n рекомендаций для каждого юзера в словарь
    top_n_df = get_top_n(predictions, n_pred=n_pred)

    return top_n_df

In [26]:
# Функция для вывода топ-N рекомендаций для каждого пользователя
def get_top_n(predictions: list, n_pred=10) -> dict:
    """Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n_pred(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    """

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n_pred]

    return top_n

In [13]:
def rec_to_df(top_n: dict) -> pd.DataFrame:
    """

    :param top_n:
    :return:
    """
    # Сохраняем всё в один датафрейм
    recomend_rate_df = pd.DataFrame()
    for elem in top_n.keys():
        rate_df = pd.DataFrame(top_n[elem],
                               columns=['Номенлатура', 'Оценка'])
        rate_df['Клиент'] = elem
        recomend_rate_df = pd.concat([recomend_rate_df, rate_df])
    # расставляем колонки в нужном порядке
    recomend_rate_df = recomend_rate_df[['Клиент', 'Номенлатура', 'Оценка']]
    return recomend_rate_df

In [4]:
raw_data = pd.read_csv('../data/raw/market_sales.csv', 
                       header=0, 
                       names=['period', 'user_id', 'store_id', 'item_id', 'license', 'type_by_nomenclature', 'rating'], 
                       dtype={'user_id':np.str, 
                              'store_id':np.str, 
                              'item_id':np.str, 
                              'license':np.int8, 
                              'type_by_nomenclature':np.str, 
                              'rating':np.int32})

In [5]:
raw_data.head(10)

,period,user_id,store_id,item_id,license,type_by_nomenclature,rating
0,2019-02-01 19:58:36,3000004608438,IZ-000034,00047364,0,C,3
1,2019-02-01 19:58:36,3000004608438,IZ-000034,00152527,0,C,3
2,2019-02-01 19:58:36,3000004608438,IZ-000034,00152528,0,C,3
3,2019-02-01 19:58:36,3000004572289,IZ-000034,00041576,1,A,4
4,2019-02-01 19:58:36,3000003024611,IZ-000034,00041577,1,B,4
5,2019-02-01 19:58:36,3000004608438,IZ-000034,00031559,0,A,3
6,2019-02-01 19:58:36,3000000634691,IZ-000034,00026724,0,B,4
7,2019-02-01 19:58:36,3000003024710,IZ-000034,00127225,0,C,3
8,2019-02-01 19:58:36,3000004608438,IZ-000034,00000763,0,C,3
9,2019-02-01 19:58:36,3000003474157,IZ-000034,00009492,0,B,3


In [85]:
store_n = 55  # позиция магазина в списке ранжированном по кол-ву продаж
n_buy = 5  # минимальное кол-во покупок клиента, для прохождения в алгоритм

In [88]:
light_data = lighter_data(raw_data, store_n=store_n, n_buy=n_buy)
data_prep_df = data_preparation(light_data)

# Генерируем top-n рекомендаций для заданных клиентов и номенклатур
top_n = make_rec(data_prep_df,
                 n_pred=10,
                 gridsearch=False,
                 alg=SVD,
                 opt_by='rmse')
# Преобразуем словарь с рекомендациями в таблицу DataFrame
top_n = rec_to_df(top_n)

print(top_n)


--------------------------------------------------------------------------
Подготовка датафрейма
--------------------------------------------------------------------------
Данные по магазину, занимающему 55-е место по кол-ву продаж, собраны
--------------------------------------------------------------------------
Подготовка trainset
--------------------------------------------------------------------------
Проводится Кроссвалидация
--------------------------------------------------------------------------
Evaluating RMSE, MAE of algorithm SVD on 4 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Mean    Std     
RMSE (testset)    0.5611  0.5689  0.5371  0.5681  0.5588  0.0129  
MAE (testset)     0.3894  0.3987  0.3672  0.3926  0.3870  0.0119  
Fit time          0.30    0.30    0.29    0.31    0.30    0.01    
Test time         0.02    0.01    0.01    0.01    0.01    0.00    
--------------------------------------------------------------------------
Генерация рекомендаций
-

In [1]:
def user_license(light_data):
    """

    :param light_data:
    :return:
    """
    user_license_df = light_data[['user_id', 'Лицензия']]

    all_user_list = user_license_df[['user_id']].drop_duplicates()
    user_with_license_list = user_license_df[user_license_df['Лицензия'] == 1].drop_duplicates()

    # соединяем список всех юзеров со списком юзеров с лицензиями
    user_license_list = all_user_list.merge(user_with_license_list, how='left', on='user_id')
    # у юзеров без лицензии появится nan, заполняем нолями
    user_license_list = user_license_list.fillna(0)
    user_license_list['Лицензия'] = user_license_list['Лицензия'].astype(int)
    # Переименовываем колонку для лучшей интерпретации
    user_license_list = user_license_list.rename(columns={'Лицензия': 'ЛицензияКлиента'})

    return user_license_list


def make_readable_top_n(top_n, light_data, list_unique_nomen):
    """

    :param top_n:
    :param light_data:
    :param list_unique_nomen:
    :return:
    """
    # добавляем название номенклатуры
    top_n_with_nom = top_n.merge(list_unique_nomen, how='left', on='КодНоменклатуры')
    # Определяем клиентов с лицензией
    user_license_list = user_license(light_data)
    # соединяем всё в один датафрейм
    top_n_readable = top_n_with_nom.merge(user_license_list, how='left', on='user_id')
    # раставляем колонки в более удобном порядке
    top_n_readable = top_n_readable[['user_id', 'КодНоменклатуры', 'Номенклатура', 'Оценка', 'ЛицензияКлиента']]
    return top_n_readable